In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time
import acquire

In [ ]:
time.sleep(#): print()

In [ ]:
link = 'https://github.com/search?p=1&q=bot&type=Repositories'

# make the request using the link of the first article and save html response to a variable
response = requests.get(link, headers={'user-agent': 'codeup data science hopper cohort'})
html = response.text
html

In [ ]:
soup = BeautifulSoup(html)
print(soup.prettify())

In [ ]:
# articles = soup.find_all('data-hydro-click')
# articles = soup.select('.v-align-middle')['href']
articles = soup.select('a.v-align-middle')
articles

In [ ]:
repos = [article['href'] for article in articles]
repos

In [ ]:
url = 'https://codeup.com/data-science/math-in-data-science/'
headers = {'User-Agent': 'Codeup Data Science'} # Some websites don't accept the pyhon-requests default user-agent
response = get(url, headers=headers)

In [ ]:
collector = []

link = 'https://github.com/search?p={i}&q=bot&type=Repositories'

for i in range(1,21):
    response = requests.get(link, headers={'user-agent': 'codeup data science hopper cohort'})
    print(response.status_code)
    html = response.text
    soup = BeautifulSoup(html)
    repos = soup.select('a.v-align-middle')
    repo_links = [repo['href'] for repo in repos]
    collector += repo_links
    

    if i % 5 == 0:
        time.sleep(60)
        
collector

In [ ]:
collector

In [ ]:
len(collector)

In [ ]:
accumulator = []
for item in collector:
    accumulator.append(item[1:])
accumulator

In [2]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
        
    except UnicodeDecodeError:
        return False
    else:
        return True